In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/content/drive/MyDrive/22-23 CMU/10-715 ML /simplicitybiaspitfalls')
sys.path.append('/content/drive/MyDrive/22-23 CMU/10-715 ML /simplicitybiaspitfalls/scripts')
from imports import *

DEVICE_ID = 0 # GPU_ID or None (CPU)



pycuda not available


In [3]:
def draw_result_lsn(model, data, num_dims=10, title=""):
    """
    Two-dimensional projection of the model's decision boundary
    onto the two most influential coordinates (measured using randomized AUC)
    """
    print ("Coordinate-wise randomized accuracies:")
    ax = lms_utils.visualize_boundary(model, data, 0, 1, num_dims, ax=None, width=1, unif_k=True, k=100_000, print_info=True)
    utils.update_ax(ax, title, 'linear noisy', 'Most influential 7-Slab', 
                    title_fs=16, label_fs=14, legend_loc=False)
    sns.despine(ax=ax, bottom=True, left=True); ax.set_xticks([]); ax.set_yticks([])
    ax.figure.savefig('/content/drive/MyDrive/22-23 CMU/10-715 ML /simplicitybiaspitfalls/figures/'+title+'.png')


In [17]:
def lsn(num_lin=1, num_slabs7=1, num_gauss=7, num_unif=0):
    num_dims=num_lin+num_slabs7+num_gauss+num_unif
    c = config =  {
        'num_train': 40_000, # training dataset size
        'dim': num_dims, # input dimension, originally 50
        'lin_margin': 0.1, # linear margin
        'slab_margin': 0.1, # slab margin,
        'same_margin': True, # keep same margin
        'random_transform': True, # apply random transform
        'width': 1, # data width
        'num_lin': num_lin, # number of linear components
        'num_slabs': 0, #. number of 5slabs
        'num_slabs7': num_slabs7, # number of 7slabs, originally 49
        'num_slabs3': 0, # number of 3slabs
        'num_gauss' : num_gauss, # number of gaussian noise coordinates
        'bs': 256, # batch size
        'corrupt_lin': 0.1, # p_noise (10% default)
        'corrupt_lin_margin': True, # noise model
        'corrupt_slab': 0.0, # slab corruption
        'num_test': 10_000, # test dataset size
        'hdim': 300, # model width
        'hl': 2, # model depth
        'mtype': 'fcn', # model architecture
        'device': gu.get_device(DEVICE_ID), # GPU device
        'lr': 0.1, # step size
        'weight_decay': 5e-4, # weight decay,
        'momentum': 0.9 # momentum
    }

    data = get_data(**c)

    # training FCN on noisy LMS data 
    model = synth_models.get_fcn(idim=c['dim'], hdim=c['hdim'], odim=2, hl=c['hl']).to(c['device'])
    opt = optim.SGD(model.parameters(), lr=c['lr'], momentum=c['momentum'], weight_decay=c['weight_decay'])

    stats = utils.fit_model(model, F.cross_entropy, opt, data['tr_dl'], 
                            data['te_dl'], gap=2_000, device=c['device'],
                            epsilon=0.005, patience_updates=1)

    # note: if it has almost converged, you can interrupt and enter Q to 
    # stop training or interrupt and change the LR to meet the stopping criterion

    # Pick model that performs best on validation data
    model = stats['best_model'][-1].to(c['device'])

    # Standard and randomized accuracies of the trained model
    s_dl = lms_utils.get_randomized_loader(data['te_dl'], data['W'], [0])
    sc_dl = lms_utils.get_randomized_loader(data['te_dl'], data['W'], list(range(1, c['dim'])))

    acc_func = utils.compute_loss_and_accuracy_from_dl
    train_accuracy, _ = acc_func(data['tr_dl'], model, F.cross_entropy, device=c['device'])
    test_accuracy, _ = acc_func(data['te_dl'], model, F.cross_entropy, device=c['device'])
    lin_accuracy, _ = acc_func(s_dl, model, F.cross_entropy, device=c['device'])
    slab_accuracy, _ = acc_func(sc_dl, model, F.cross_entropy, device=c['device'])

    print ('Train Accuracy: {:.3f}'.format(train_accuracy.item()))
    print ('Test Accuracy: {:.3f}'.format(test_accuracy.item()))
    print ('Linear (S) Randomized Accuracy: {:.3f}'.format(lin_accuracy.item()))
    print ('Slab (S^c) Randomzied Accuracy: {:.3f}'.format(slab_accuracy.item()))
    print ('=================================================')
    
    title="{}+{}+{}+{}={}".format(num_lin, num_slabs7, num_gauss, num_unif, num_dims)
    draw_result_lsn(model=model, data=data, num_dims=num_dims, title=title)
    return {
        'input': config,
        'output': {
            'Train Accuracy' : train_accuracy.item(),
            'Test Accuracy': test_accuracy.item(),
            'Linear (S) Randomized Accuracy': lin_accuracy.item(),
            'Slab (S^c) Randomzied Accuracy':slab_accuracy.item(),
        }
    }

In [ ]:
import torch
torch.manual_seed(17)
import random
random.seed(17)
import numpy as np
np.random.seed(17)

for num_unif in range(34,35):
  result = lsn(num_lin=1, num_slabs7=1, num_gauss=0, num_unif=num_unif)
  with open('/content/drive/MyDrive/22-23 CMU/10-715 ML /simplicitybiaspitfalls/data.txt', 'a') as f:
    f.write(str(result) + '\n')

accuracy/loss measured every 2000 updates
update_print_gap: 2000, epss: 0.005, bs: 256, device: cuda:0
#updates, train acc, test acc, train loss
0 0.4668 0.4645 0.6957
2000 0.9536 0.9464 0.0818
4000 0.9552 0.9466 0.0825
6000 0.9549 0.9449 0.0830
8000 0.9549 0.9463 0.0844
10000 0.9564 0.9470 0.0760
12000 0.9458 0.9401 0.1041
14000 0.9575 0.9462 0.0769
16000 0.9564 0.9468 0.0758
18000 0.9571 0.9470 0.0752
20000 0.9567 0.9462 0.0818
22000 0.9567 0.9444 0.0775
24000 0.9585 0.9485 0.0759
26000 0.9574 0.9497 0.0757
28000 0.9555 0.9464 0.0790
30000 0.9574 0.9472 0.0785
32000 0.9562 0.9451 0.0800
34000 0.9583 0.9472 0.0753
36000 0.9585 0.9459 0.0744
38000 0.9567 0.9470 0.0812
40000 0.9584 0.9486 0.0746
42000 0.9577 0.9460 0.0769
44000 0.9562 0.9472 0.0815
46000 0.9570 0.9469 0.0775
48000 0.9581 0.9473 0.0761
50000 0.9575 0.9462 0.0808
52000 0.9588 0.9464 0.0775
54000 0.9578 0.9472 0.0747
56000 0.9581 0.9456 0.0780
58000 0.9578 0.9463 0.0789
60000 0.9553 0.9479 0.0786
62000 0.9576 0.9445 0.0822

'[{\'input\': {\'num_train\': 40000, \'dim\': 9, \'lin_margin\': 0.1, \'slab_margin\': 0.1, \'same_margin\': True, \'random_transform\': True, \'width\': 1, \'num_lin\': 1, \'num_slabs\': 0, \'num_slabs7\': 1, \'num_slabs3\': 0, \'num_gauss\': 7, \'bs\': 256, \'corrupt_lin\': 0.1, \'corrupt_lin_margin\': True, \'corrupt_slab\': 0.0, \'num_test\': 10000, \'hdim\': 300, \'hl\': 2, \'mtype\': \'fcn\', \'device\': "device(type=\'cuda\', index=0)", \'lr\': 0.1, \'weight_decay\': 0.0005, \'momentum\': 0.9}, \'output\': {\'Train Accuracy\': 0.952075, \'Test Accuracy\': 0.9504, \'Linear (S) Randomized Accuracy\': 0.4843, \'Slab (S^c) Randomzied Accuracy\': 0.9501}}]'